In [97]:
import numpy as np
from sklearn.linear_model import LassoCV
import utils
from CoRT_builder import CoRT
import importlib
importlib.reload(utils)

<module 'utils' from 'D:\\Data Science Project\\CorT SI\\utils.py'>

In [98]:
import utils
import importlib
importlib.reload(utils)

<module 'utils' from 'D:\\Data Science Project\\CorT SI\\utils.py'>

In [2]:
# Start CorT
n_target = 100
n_source = 200
p = 500
K = 10
Ka = 8
h = 30
lamda = 0.1
s_vector = [0.4, 0.4, 0.4, 0.5, 0.5, 0.5, -0.6, -0.6, -0.6, -0.6]
T = 5
s = len(s_vector)
CoRT = CoRT()
target_data, source_data = CoRT.gen_data(n_target, n_source, p, K, Ka, h, s_vector, s, "AR")
similar_source_index = CoRT.find_similar_source(n_target, K, target_data, source_data, T=5)

X_combined, y_combined = CoRT.prepare_CoRT_data(similar_source_index, source_data, target_data)

model = LassoCV(cv=5, fit_intercept=True, random_state=42, n_jobs=-1)
model.fit(X_combined, y_combined.ravel())
beta_hat_target = model.coef_[-p:]

# SI
active_beta_hat_target = []
for b in beta_hat_target:
  if b != 0:
    active_beta_hat_target.append(b)

j = np.random.choice(np.arange(0, len(active_beta_hat_target)))

Total 8 similar sources: [0, 1, 2, 3, 4, 5, 6, 7]


In [3]:
X_target = target_data["X"]
y_target = target_data["y"]
X_active, X_inactive = utils.get_active_X(beta_hat_target, X_target)
etaj, etajTy = utils.construct_test_statistic(y_target, j, X_active) # This from the Cort Model

Sigma = np.eye(n_target)
b_global = Sigma @ etaj @ np.linalg.pinv(etaj.T @ Sigma @ etaj)
a_global = (np.eye(n_target) - b_global @ etaj.T) @ y_target

In [99]:
folds= utils.split_target(T, X_target, y_target, n_target)
L_train, R_train = utils.get_Z_train(etajTy, folds, source_data, a_global, b_global, lamda, K, T)

In [104]:
print(f"L_train: {L_train}")
print(f"R_train: {R_train}")

L_train: 0.021319754320172246
R_train: 0.021807547482283758


In [103]:
etajTy

0.021336804249645975